In [2]:
!pip install findspark
!pip install pyspark
!pip install plotly

In [3]:
# Import findspark and initialize.
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_data = spark.read.option('header','true').csv(SparkFiles.get("home_sales_revised.csv"),
                                                    inferSchema = True, sep=",", timestampFormat="yyyy-mm-dd")

sales_data.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [22]:
# 2. Create a temporary view of the DataFrame.
home_sales = sales_data.createOrReplaceTempView('sales')

spark.sql("SELECT * FROM sales LIMIT 10").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [29]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from sales WHERE bedrooms == 4 GROUP BY YEAR(date) ORDER BY ROUND(AVG(price),2) DESC").show()


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           301819.44|      2021|
|            300263.7|      2019|
|           298353.78|      2020|
|           296363.88|      2022|
+--------------------+----------+



In [31]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price),2), date_built from sales WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY date_built ORDER BY ROUND(AVG(price),2) DESC").show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           295962.27|      2013|
|           293683.19|      2012|
|           292859.62|      2010|
|           292676.79|      2017|
|           291117.47|      2011|
|           290852.27|      2014|
|           290555.07|      2016|
|            288770.3|      2015|
+--------------------+----------+



In [34]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
sqrft_2000 = """
(SELECT ROUND(AVG(price),2), date_built
FROM sales WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_lot >= 2000
GROUP BY date_built ORDER BY ROUND(AVG(price),2) DESC)
"""
spark.sql(sqrft_2000).show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           295858.68|      2012|
|           295142.13|      2013|
|           294195.13|      2014|
|           291788.36|      2015|
|            287812.6|      2016|
|           282026.59|      2017|
|           281413.45|      2011|
|           280447.23|      2010|
+--------------------+----------+



In [36]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("SELECT ROUND(AVG(price),2), view FROM sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           1026669.5| 100|
|          1061201.42|  99|
|          1053739.33|  98|
|          1129040.15|  97|
|          1017815.92|  96|
|           1054325.6|  95|
|           1033536.2|  94|
|          1026006.06|  93|
|           970402.55|  92|
|          1137372.73|  91|
|          1062654.16|  90|
|          1107839.15|  89|
|          1031719.35|  88|
|           1072285.2|  87|
|          1070444.25|  86|
|          1056336.74|  85|
|          1117233.13|  84|
|          1033965.93|  83|
|           1063498.0|  82|
|          1053472.79|  81|
+--------------------+----+
only showing top 20 rows

--- 0.7705152034759521 seconds ---


In [37]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [39]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [40]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(price),2), view FROM sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           1026669.5| 100|
|          1061201.42|  99|
|          1053739.33|  98|
|          1129040.15|  97|
|          1017815.92|  96|
|           1054325.6|  95|
|           1033536.2|  94|
|          1026006.06|  93|
|           970402.55|  92|
|          1137372.73|  91|
|          1062654.16|  90|
|          1107839.15|  89|
|          1031719.35|  88|
|           1072285.2|  87|
|          1070444.25|  86|
|          1056336.74|  85|
|          1117233.13|  84|
|          1033965.93|  83|
|           1063498.0|  82|
|          1053472.79|  81|
+--------------------+----+
only showing top 20 rows

--- 0.6299500465393066 seconds ---


The uncached runtime is *0.77 seconds* while the cached runtime totals to *0.63 seconds* which is ***0.14 seconds less*** now that the data has been sorted through cache.

In [42]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_data.write.partitionBy("date_built").mode("overwrite").parquet("sales")

In [45]:
# 11. Read the formatted parquet data.
p_sales_data = spark.read.parquet("sales")


In [46]:
# 12. Create a temporary table for the parquet data.
p_sales_data.createOrReplaceTempView('p_sales')

In [47]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
spark.sql("SELECT ROUND(AVG(price),2), view FROM sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC").show()



print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           1026669.5| 100|
|          1061201.42|  99|
|          1053739.33|  98|
|          1129040.15|  97|
|          1017815.92|  96|
|           1054325.6|  95|
|           1033536.2|  94|
|          1026006.06|  93|
|           970402.55|  92|
|          1137372.73|  91|
|          1062654.16|  90|
|          1107839.15|  89|
|          1031719.35|  88|
|           1072285.2|  87|
|          1070444.25|  86|
|          1056336.74|  85|
|          1117233.13|  84|
|          1033965.93|  83|
|           1063498.0|  82|
|          1053472.79|  81|
+--------------------+----+
only showing top 20 rows

--- 0.8897640705108643 seconds ---


The the cached runtime totals to *0.63 seconds* while the parquet runtime totals to *0.89 seconds* which is ***0.26 seconds more*** now that the data has been sorted through parquet.

In [48]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [49]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("sales"):
  print("table is till cached")
else:
  print("all clear")


all clear
